In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt


file = pd.read_csv("http://ladi.s3-us-west-2.amazonaws.com/Labels/ladi_aggregated_responses_url.tsv",delimiter='\t',header='infer')
file["Answer"] = file["Answer"].str.strip('[|]')
file["Answer"] = file["Answer"].str.split(",",expand = True)

# Extract labels with damage and infrastructure categories
label_damage_infra = file[file['Answer'].str.contains('damage|infrastructure',na=False,case=False)]

#Filter out infrastructure label with label 'none'
label_clean = label_damage_infra[~label_damage_infra['Answer'].str.contains('none',na=False,case=False)]

# Extract url data with the label does not contain 'flood'
urls = label_clean['url'].unique().tolist()
labels = label_clean['Answer'].unique().tolist()

metadata = pd.read_csv('http://ladi.s3-us-west-2.amazonaws.com/Labels/ladi_images_metadata.csv')


used_metadata = metadata[metadata['url'].isin(urls)]

# Generate url and s3_path features into list
meta_lst = used_metadata['url'].tolist()
meta_s3_lst = used_metadata['s3_path'].tolist()

# Check how many images do not have metadata but have human labels
human_label_only = list(set(urls) - set(meta_lst))
print(len(human_label_only))

#landslide_tiny_lst = sample(landslide_meta_s3_lst, 831)
tiny_metadata = metadata[metadata['s3_path'].isin(meta_s3_lst)]

# creating the new datasets
labels_list = []
for label in labels:
    first = label_clean[label_clean['Answer'].str.contains(label,na=False,case=False)]['url'].unique().tolist()
    s3_series = used_metadata[used_metadata['url'].isin(first)]['s3_path']
    label_col =  pd.Series(name="label")
    df = pd.concat([s3_series, label_col], axis=1)
    df['label'] = label
    labels_list.append(df)

label_df = pd.concat(labels_list)
label_df = label_df.groupby(['s3_path']).agg('sum')

tiny_metadata.to_csv('tiny_metadata.csv')
label_df.to_csv('tiny_label.csv')

0


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:41: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.


In [2]:
df = pd.read_csv('tiny_label.csv')

In [19]:
l = []
for j in df['label'].values:
    l.append([i for i in j.split("'") if i])

In [21]:
df['label'] = l

In [24]:
df.loc[4,'s3_path']

's3://ladi/Images/FEMA_CAP/1013/20147/DSC_1575_787dc8f2-fb8f-4464-99a2-45ba5fc677c2.jpg'

In [28]:
metadata = pd.read_csv('tiny_metadata.csv')
metadata = metadata[['s3_path','url']]
metadata

,s3_path,url
0,s3://ladi/Images/FEMA_CAP/1013/20195/DSC_0512_...,https://ladi.s3-us-west-2.amazonaws.com/Images...
1,s3://ladi/Images/FEMA_CAP/1013/20195/DSC_0516_...,https://ladi.s3-us-west-2.amazonaws.com/Images...
2,s3://ladi/Images/FEMA_CAP/1013/20195/DSC_0521_...,https://ladi.s3-us-west-2.amazonaws.com/Images...
3,s3://ladi/Images/FEMA_CAP/1013/20195/DSC_0529_...,https://ladi.s3-us-west-2.amazonaws.com/Images...
4,s3://ladi/Images/FEMA_CAP/1013/20211/DSC_1028_...,https://ladi.s3-us-west-2.amazonaws.com/Images...
...,...,...
30096,s3://ladi/Images/FEMA_CAP/9168/616298/DSC_3980...,https://ladi.s3-us-west-2.amazonaws.com/Images...
30097,s3://ladi/Images/FEMA_CAP/9168/616298/DSC_3990...,https://ladi.s3-us-west-2.amazonaws.com/Images...
30098,s3://ladi/Images/FEMA_CAP/9168/616298/DSC_4000...,https://ladi.s3-us-west-2.amazonaws.com/Images...
30099,s3://ladi/Images/FEMA_CAP/9168/616298/DSC_3982...,https://ladi.s3-us-west-2.amazonaws.com/Images...


In [31]:
final_df = df.merge(metadata,how='inner',on='s3_path')

In [32]:
final_df.to_csv('finalBoi.csv')